In [1]:
import json
with open("stair_captions/stair_captions_v1.2_val.json", "r",encoding="utf-8") as f:
  json_data_val = json.load(f)
with open("stair_captions/stair_captions_v1.2_train.json", "r",encoding="utf-8") as f:
  json_data_train = json.load(f)

In [2]:
dataset = {}
ids = []
captions = []

def build_dataset(dataset, json_data):
  num_samples = len(json_data['annotations'])
  for i in range(num_samples):
    anno = json_data['annotations'][i]
    image_id = anno["image_id"]
    image_captions = dataset.get(image_id, [])
    image_captions.append((anno["id"], anno["caption"]))
    ids.append(anno["id"])
    captions.append(anno["caption"])
    dataset[image_id] = image_captions

build_dataset(dataset, json_data_val)
build_dataset(dataset, json_data_train)

In [ ]:
#dataset

In [3]:
id2idx = {id:idx for idx, id in enumerate(ids)}
#id2idx

In [5]:
len(captions)

616435

In [6]:
import numpy as np
import spacy
import pkg_resources, imp
imp.reload(pkg_resources)

#nlp = spacy.load('ja_ginza_nopn')
nlp = spacy.load("ja_ginza")

vectors = []
i = 0
for caption in captions:
    if i %1000 ==0:
        print(i)
    doc = nlp(caption, disable=['ner'])
    vectors.append(doc.vector)
    i = i +1
'''
'''
'''
import multiprocessing as mp
pool = mp.Pool(mp.cpu_count())
def count_sim(caption):
    #if i %1000 ==0:
    print(i)
    doc = nlp(caption, disable=['ner'])
    i = i +1
    return doc.vector
results = [pool.apply(count_sim, args=(caption)) for caption in captions]
#results = [pool.map(count_sim,captions)]  
'''

del nlp

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
def similarity(id1, id2):
  return cosine_similarity([vectors[id2idx[id1]]], [vectors[id2idx[id2]]])[0][0]

In [8]:
import random

def make_triplets(dataset, threshold=0.85, seed=7, max_tries=25):
  triplets = []
  random.seed(seed)
  neg_candidate_indices = list(range(len(ids)))
  random.shuffle(neg_candidate_indices)
  def log(i, str):
    if i % 5000 == 0:
      print(str)

  for i, image_id in enumerate(list(dataset.keys())): 
    log(i, "### %d ###" % (image_id))

    # pickup positive pair.
    score = 0.0
    tries = 0
    while score < threshold : 
      [(id, caption), (id_pos, caption_pos)] = random.sample(dataset[image_id],2)
      score = similarity(id, id_pos)
      tries+=1
      if tries > max_tries:
        break
    if score < threshold:
      continue

    # pickup negative one.
    id_neg = id
    current_caption_ids = [id_cap[0] for id_cap in dataset[image_id]]
    while id_neg in current_caption_ids:
      idx_neg = neg_candidate_indices.pop()
      id_neg = ids[idx_neg]
    caption_neg = captions[id2idx[id_neg]]

    log(i, "  pos:  score: %4.2f [%s]:[%s]" % (score, caption, caption_pos))
    log(i, "  neg:  score: %4.2f [%s]:[%s]" % (similarity(id, id_neg), caption, caption_neg))
    triplets.append({
      "image_id": image_id,
      "id": id,
      "id_pos": id_pos,
      "id_neg": id_neg,
      "caption": caption,
      "caption_pos": caption_pos,
      "caption_neg": caption_neg  
    })
  return triplets

In [9]:
triplets = make_triplets(dataset)

### 580856 ###
  pos:  score: 0.86 [夜で、まだオープンしている場所の入り口]:[踏切の近くにワイナリーが開店している]
  neg:  score: 0.80 [夜で、まだオープンしている場所の入り口]:[野球選手のフォームの確認をしている]
### 235486 ###
  pos:  score: 0.91 [2人の女性と1人の男性が食卓を囲んでいる]:[三人の男女がピザと飲み物を囲んで笑っている]
  neg:  score: 0.85 [2人の女性と1人の男性が食卓を囲んでいる]:[ストックを使って子どもをひっぱってスキーをしている人]
### 133631 ###
  pos:  score: 0.97 [柵の手前に土の上に立った象が1頭いる]:[1頭のゾウが柵の内側に立っている]
  neg:  score: 0.78 [柵の手前に土の上に立った象が1頭いる]:[スケートボードに乗った少年がジャンプしている]
### 467843 ###
  pos:  score: 0.89 [メガネをかけた少年がフリスビーを見つめている]:[フリスビーで遊んでいる子どもだ]
  neg:  score: 0.91 [メガネをかけた少年がフリスビーを見つめている]:[黒い犬が芝生の上をフリスビーを加えながら走っている]
### 423588 ###
  pos:  score: 0.93 [雪山の小屋の屋根に登る人と撮影する人]:[雪山の小さな小屋の屋根に、スノーボードを履いた人物が乗り、それを3人の人物が見ている]
  neg:  score: 0.73 [雪山の小屋の屋根に登る人と撮影する人]:[とっても大きくてきれいな時計台がある]
### 115765 ###
  pos:  score: 0.92 [芝生の木立の中で番傘をさした女とギターケースを背負った男が話をしている]:[ギターを背負った男性と、民族衣装を着て傘をさした女性がいる]
  neg:  score: 0.84 [芝生の木立の中で番傘をさした女とギターケースを背負った男が話をしている]:[列車が停車している駅のホームに向かって線路を横切っている人がいる]
### 538330 ###
  pos:  score: 0.92 [マットの上に料理と飲み物が置いてあ

In [10]:
len(triplets)

122810

In [13]:
import pandas as pd
dt = pd.DataFrame(triplets)
dt

,image_id,id,id_pos,id_neg,caption,caption_pos,caption_neg
0,580856,812634,4,277176,夜で、まだオープンしている場所の入り口,踏切の近くにワイナリーが開店している,野球選手のフォームの確認をしている
1,263834,246909,193567,652313,渋滞で二人乗りのバイクが前に進めず止まっている,黒いヘルメットを被った子供がバイクの後ろに乗っている,座っている男女、女性はスマートフォンを操作している
2,225784,310939,7,428510,室内で鏡の前にて折りたたみ自転車がスタンドを立てて立っている,部屋の中に白い自転車が置いてある,全体に白く、一部後ろに赤が入った飛行機が飛んでいる
3,353807,203228,296182,208344,白い荷台を引く白い馬がこちらを見ている,白い馬の引くキャリッジが建物の前にいる,濡れた路面の上をオートバイに乗った人々が走っている
4,133149,672044,13,313116,海岸でカモメが翼を広げて羽ばたいている,ビーチ近くの水際に鳥が羽を広げて停まっている,真ん中にはひょうたん型の鉢がある
...,...,...,...,...,...,...,...
122805,522616,828265,835794,680595,このキッチンには沢山の道具がそろっている,食器が多く並んだ古いキッチンがある,白と黒の模様の猫が座っている
122806,398142,830905,826455,433747,茶色いと白の建物の手前にあるのは青信号である,建物の前の広場に多くの人がいる,自動車のハンドルの上に手のひらサイズの猫の人形が乗っている
122807,185632,825871,835275,555388,白い石の暖炉のある絨毯の引いてある部屋,白い壁とフローリングと絨毯がある,ライトを点けたバスが路肩に停車している
122808,322530,826028,835523,819684,列車の外側はほとんど赤で塗られている,赤い列車の連結部分の扉が開いている,STOPの赤い標識や文字が書かれた紙を持つ手


In [14]:
import joblib
joblib.dump(dt,"texts_orinal_data.pkl")

['texts_orinal_data.pkl']

In [15]:
joblib.load("texts_orinal_data.pkl")

,image_id,id,id_pos,id_neg,caption,caption_pos,caption_neg
0,580856,812634,4,277176,夜で、まだオープンしている場所の入り口,踏切の近くにワイナリーが開店している,野球選手のフォームの確認をしている
1,263834,246909,193567,652313,渋滞で二人乗りのバイクが前に進めず止まっている,黒いヘルメットを被った子供がバイクの後ろに乗っている,座っている男女、女性はスマートフォンを操作している
2,225784,310939,7,428510,室内で鏡の前にて折りたたみ自転車がスタンドを立てて立っている,部屋の中に白い自転車が置いてある,全体に白く、一部後ろに赤が入った飛行機が飛んでいる
3,353807,203228,296182,208344,白い荷台を引く白い馬がこちらを見ている,白い馬の引くキャリッジが建物の前にいる,濡れた路面の上をオートバイに乗った人々が走っている
4,133149,672044,13,313116,海岸でカモメが翼を広げて羽ばたいている,ビーチ近くの水際に鳥が羽を広げて停まっている,真ん中にはひょうたん型の鉢がある
...,...,...,...,...,...,...,...
122805,522616,828265,835794,680595,このキッチンには沢山の道具がそろっている,食器が多く並んだ古いキッチンがある,白と黒の模様の猫が座っている
122806,398142,830905,826455,433747,茶色いと白の建物の手前にあるのは青信号である,建物の前の広場に多くの人がいる,自動車のハンドルの上に手のひらサイズの猫の人形が乗っている
122807,185632,825871,835275,555388,白い石の暖炉のある絨毯の引いてある部屋,白い壁とフローリングと絨毯がある,ライトを点けたバスが路肩に停車している
122808,322530,826028,835523,819684,列車の外側はほとんど赤で塗られている,赤い列車の連結部分の扉が開いている,STOPの赤い標識や文字が書かれた紙を持つ手
